<center><font size="10"> Problems And Issues In Neural Networks </font></center>

In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

#### The backpropagation algorithm works by going from the output layer to the input layer, propagating the error gradient on the way.
#### Unfortunately, gradients often get smaller and smaller as the algorithm progresses down to the lower layers. As a result, the Gradient Descent update leaves the lower layer connection weights  virtually unchanged, and training never converges to a good solution. This is called the **VANISHING GRADIENTS** problem. In some cases, the opposite can happen: the gradients can grow bigger and bigger, so many layers get insanely large weight updates and the algorithm diverges. This is the **EXPLODING GRADIENTS** problem, which is mostly encountered in recurrent neural networks.

#### To solve the above problem we need the **variance of the outputs of each layer to be equal to the variance of its inputs** and we also need the gradients to have **equal variance before and after flowing through a layer in the reverse direction**. It is actually not possible to guarantee both unless the layer has an equal number of inputs and neurons (these numbers are called the **fan-in and fan-out** of the layer)

#### To over come this we use **Xavier initialization** or **He initialization**

In [4]:
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

#### He initialization with a uniform distribution, but based on $ fan_{avg} $ rather than $ fan_{in} $, we can use the VarianceScaling initializer like this

In [5]:
init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg',
                                        distribution='uniform')

keras.layers.Dense(10, activation="relu", kernel_initializer=init)

### Nonsaturating Activation Functions

#### The ReLU activation function suffers from a problem known as the **dying ReLUs**: during training, some neurons effectively die, meaning they stop outputting anything other than 0. In some cases, you may find that half of your network’s neurons are dead, especially if you used a large learning rate.

#### To solve this problem, you may want to use a variant of the ReLU function, such as the **leaky ReLU** or **randomized leaky ReLU (RReLU)** or **parametric leaky ReLU (PReLU)**. Sole difference in the 3 being difference in α values. Also u can use **exponential linear unit (ELU)** and **Scaled ELU (SELU)**

#### Lets train Fashion MNIST on leaky ReLU and PReLU and also SELU

#### ReLU

In [6]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [9]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [10]:
model.compile(loss="sparse_categorical_crossentropy",
            optimizer=keras.optimizers.SGD(learning_rate=1e-3),
            metrics=["accuracy"])

In [11]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 11s 5ms/step - loss: 1.2641 - accuracy: 0.6143 - val_loss: 0.8490 - val_accuracy: 0.7210
Epoch 2/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.7702 - accuracy: 0.7413 - val_loss: 0.6926 - val_accuracy: 0.7738
Epoch 3/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.6655 - accuracy: 0.7796 - val_loss: 0.6335 - val_accuracy: 0.7920
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.6098 - accuracy: 0.7992 - val_loss: 0.5810 - val_accuracy: 0.8130
Epoch 5/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5736 - accuracy: 0.8115 - val_loss: 0.5516 - val_accuracy: 0.8220
Epoch 6/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5470 - accuracy: 0.8181 - val_loss: 0.5293 - val_accuracy: 0.8274
Epoch 7/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5269 - accuracy: 0.8232 - val_loss: 0.5111 - val_accuracy

#### PReLU

In [12]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [13]:
model.compile(loss="sparse_categorical_crossentropy",
            optimizer=keras.optimizers.SGD(learning_rate=1e-3),
            metrics=["accuracy"])

In [14]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 11s 6ms/step - loss: 1.3628 - accuracy: 0.5709 - val_loss: 0.9141 - val_accuracy: 0.7040
Epoch 2/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.8204 - accuracy: 0.7324 - val_loss: 0.7274 - val_accuracy: 0.7660
Epoch 3/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.6985 - accuracy: 0.7739 - val_loss: 0.6547 - val_accuracy: 0.7932
Epoch 4/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.6340 - accuracy: 0.7933 - val_loss: 0.5974 - val_accuracy: 0.8120
Epoch 5/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.5918 - accuracy: 0.8061 - val_loss: 0.5623 - val_accuracy: 0.8236
Epoch 6/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.5614 - accuracy: 0.8136 - val_loss: 0.5373 - val_accuracy: 0.8284
Epoch 7/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.5385 - accuracy: 0.8201 - val_loss: 0.5162 - val_ac

#### Lazy ReLU ==> 8s 5ms/step - loss: 0.4880 - accuracy: 0.8347 - val_loss: 0.4806 - val_accuracy: 0.8414
####   PReLU   ==> 10s 6ms/step - loss: 0.4935 - accuracy: 0.8319 - val_loss: 0.4795 - val_accuracy: 0.8400

#### All though we cannot see a difference except for the time taken but in complex conditions they do affect


#### SELU

In [15]:
np.random.seed(42)
tf.random.set_seed(42)
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu",
                            kernel_initializer="lecun_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu",
                                kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [16]:
model.compile(loss="sparse_categorical_crossentropy",
            optimizer=keras.optimizers.SGD(learning_rate=1e-3),
            metrics=["accuracy"])

#### Scaling the Inputs for SELU

In [17]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [18]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/5
1719/1719 [==============================] - 69s 38ms/step - loss: 1.3340 - accuracy: 0.4734 - val_loss: 0.9590 - val_accuracy: 0.6292
Epoch 2/5
1719/1719 [==============================] - 67s 39ms/step - loss: 0.9277 - accuracy: 0.6541 - val_loss: 0.8934 - val_accuracy: 0.6664
Epoch 3/5
1719/1719 [==============================] - 67s 39ms/step - loss: 0.7940 - accuracy: 0.7074 - val_loss: 0.7187 - val_accuracy: 0.7358
Epoch 4/5
1719/1719 [==============================] - 62s 36ms/step - loss: 0.8667 - accuracy: 0.6788 - val_loss: 0.9225 - val_accuracy: 0.6914
Epoch 5/5
1719/1719 [==============================] - 65s 38ms/step - loss: 0.8008 - accuracy: 0.7124 - val_loss: 0.7559 - val_accuracy: 0.7374


#### Now lets try using ReLU

In [19]:
np.random.seed(42)
tf.random.set_seed(42)
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [20]:
model.compile(loss="sparse_categorical_crossentropy",
            optimizer=keras.optimizers.SGD(learning_rate=1e-3),
            metrics=["accuracy"])

In [21]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/5
1719/1719 [==============================] - 70s 39ms/step - loss: 1.8299 - accuracy: 0.2372 - val_loss: 1.6775 - val_accuracy: 0.2398
Epoch 2/5
1719/1719 [==============================] - 63s 36ms/step - loss: 1.2152 - accuracy: 0.4655 - val_loss: 0.9297 - val_accuracy: 0.6156
Epoch 3/5
1719/1719 [==============================] - 63s 37ms/step - loss: 0.9313 - accuracy: 0.6097 - val_loss: 0.9417 - val_accuracy: 0.6004
Epoch 4/5
1719/1719 [==============================] - 60s 35ms/step - loss: 0.8036 - accuracy: 0.6684 - val_loss: 0.7295 - val_accuracy: 0.7086
Epoch 5/5
1719/1719 [==============================] - 62s 36ms/step - loss: 0.8346 - accuracy: 0.6641 - val_loss: 0.7133 - val_accuracy: 0.7140


#### Much worse accuracy, we suffered from the vanishing/exploding gradients problem.

#### Using He initialization along with ELU (or any variant of ReLU) can significantly reduce the vanishing/exploding gradients problems at the beginning of training, it doesn’t guarantee that they won’t come back during training.
#### To ensure the the problems do not come back during training we have to use **Batch Normalization**

## Batch Normalization

In [23]:
model = keras.models.Sequential([
        keras.layers.Flatten(input_shape = [28, 28]),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(300, activation = 'elu', kernel_initializer = 'he_normal'),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(100, activation = "elu", kernel_initializer = 'he_normal'),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(10, activation = 'softmax')
        ])

In [24]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_2 (Batc  (None, 784)              3136      
 hNormalization)                                                 
                                                                 
 dense_211 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_3 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_212 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_4 (Batc  (None, 100)             

In [25]:
model.compile(loss= "sparse_categorical_crossentropy",
                optimizer = keras.optimizers.SGD(learning_rate=1e-3),
                metrics = ['accuracy'])

In [26]:
history = model.fit(X_train, y_train, epochs = 10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 17s 9ms/step - loss: 0.8829 - accuracy: 0.7025 - val_loss: 0.5841 - val_accuracy: 0.8018
Epoch 2/10
1719/1719 [==============================] - 16s 10ms/step - loss: 0.5951 - accuracy: 0.7939 - val_loss: 0.5076 - val_accuracy: 0.8320
Epoch 3/10
1719/1719 [==============================] - 16s 9ms/step - loss: 0.5427 - accuracy: 0.8103 - val_loss: 0.4722 - val_accuracy: 0.8436
Epoch 4/10
1719/1719 [==============================] - 16s 9ms/step - loss: 0.5075 - accuracy: 0.8237 - val_loss: 0.4507 - val_accuracy: 0.8512
Epoch 5/10
1719/1719 [==============================] - 17s 10ms/step - loss: 0.4867 - accuracy: 0.8297 - val_loss: 0.4355 - val_accuracy: 0.8560
Epoch 6/10
1719/1719 [==============================] - 17s 10ms/step - loss: 0.4711 - accuracy: 0.8349 - val_loss: 0.4237 - val_accuracy: 0.8592
Epoch 7/10
1719/1719 [==============================] - 17s 10ms/step - loss: 0.4595 - accuracy: 0.8374 - val_loss: 0.4126 - va

#### Another technique to mitigate the exploding gradients problem is to simply clip the gradients during backpropagation so that they never exceed some threshold. This is called **Gradient Clipping**.

### Gradient Clipping

In [27]:
optimizer = keras.optimizers.SGD(clipvalue = 1.0)
model.compile(loss = 'mse', optimizer=optimizer)

#### OR

In [ ]:
optimizer = keras.optimizers.SGD(clipnorm=1.0)
model.compile(loss = 'mse', optimizer=optimizer)